In [1]:
from typing import NamedTuple, Optional 

class MQI(NamedTuple):
    TC: float
    TN: float
    TP: float
    TK: float
    
    @property
    def CN_ratio(self):
        return self.TC / self.TN
    

In [2]:
MQIs = {"cow": MQI(TC=320000, TN=18000, TP=8000, TK=21000),
        "chicken": MQI(TC=250000, TN=30000, TP=18000, TK=28000),
        "soy": MQI(TC=360000, TN=60000, TP=12000, TK=25000)
       }

In [3]:
class Range(NamedTuple):
    min: Optional[float]
    max: Optional[float]

RecomendedValues = {
    "Bacteria Density": Range(2.0, None),
    "TC": Range(12000, None),
    "TN": Range(1000, None),
    "TP": Range(1100, None),
    "TK": Range(2500, 10000),
    "C/N": Range(8, 27),
    "N-activity": Range(25, None),
    "P-activity": Range(20, 80)
}

max_manure_amount = 5000 / (10 * 5000 * 4.5)

RecomendedValues["Amount Manure(kg)"] = Range(None, max_manure_amount)

max_organic_amount = 500 / (10 * 5000 * 4.5)

RecomendedValues["Amount Organic(kg)"] = Range(None, max_organic_amount)

In [4]:
import pandas as pd
import numpy as np

In [14]:
MQIs = pd.DataFrame(
    {"cow manure": {"TC": 320000, "TN": 18000, "TP": 8000, "TK": 21000, "C/N": 17.8, "Fermented": True},
     "chicken manure": {"TC": 250000, "TN": 30000, "TP": 18000, "TK": 28000, "C/N": 8.3, "Fermented": True},
     "soy meal": {"TC": 360000, "TN": 60000, "TP": 12000, "TK": 25000, "C/N": 6.0, "Fermented": False},
     "rice bran": {"TC": 350000, "TN": 20000, "TP": 35000, "TK": 20000, "C/N": 17.5, "Fermented": False},
     "fish bone": {"TC": 320000, "TN": 70000, "TP": 60000, "TK": 10000, "C/N": 4.6, "Fermented": False},
     "bone": {"TC": 210000, "TN": 50000, "TP": 100000, "TK": 1500, "C/N": 4.2, "Fermented": False}
       }).transpose()

In [15]:
MQIs

,TC,TN,TP,TK,C/N,Fermented
cow manure,320000,18000,8000,21000,17.8,True
chicken manure,250000,30000,18000,28000,8.3,True
soy meal,360000,60000,12000,25000,6.0,False
rice bran,350000,20000,35000,20000,17.5,False
fish bone,320000,70000,60000,10000,4.6,False
bone,210000,50000,100000,1500,4.2,False


In [16]:
selected_fields = ["TC", "TN", "TP", "TK"]


def calc_TX(TX_old, mqi, increase):
    return (TX_old + increase * mqi)/(1 + increase)


def calc_min_required(sofix):
    min_vals = np.array([RecomendedValues[fld].min for fld in selected_fields])

    sofix_selected = np.array([sofix[fld] for fld in selected_fields])

#     mqi = MQIs[selected_fields].loc["cow"].to_numpy()

    for x in MQIs.iterrows():
        print(20*"*")
        print(x[0])
#         print(MQIs.loc[x[0]])
        mqi = x[1][selected_fields].to_numpy()
        mins_required = (min_vals - sofix_selected)/(mqi - min_vals)
        print(f"  {mins_required=}")
        min_required = max(mins_required)
        print(f"  {min_required=} | maximum allowed {RecomendedValues['Amount Manure(kg)'].max}")
        new_TXs = calc_TX(sofix_selected, mqi, min_required)
        display_fields = selected_fields + ["C/N"]
        new_TXs = np.append(new_TXs, new_TXs[0]/new_TXs[1])
        df = pd.DataFrame({k: [v] for k, v in zip(display_fields, new_TXs)})
        print(f"{df}")
#         print(f"  C/N:{new_TXs[0]/new_TXs[1]}")

In [17]:
sofix1 = {"TC": 11000, "TN": 800, "TP": 900, "TK": 2500, "C/N": 13.8}

calc_min_required(sofix1)

********************
cow manure
  mins_required=array([0.003246753246753247, 0.011764705882352941, 0.028985507246376812,
       0.0], dtype=object)
  min_required=0.028985507246376812 | maximum allowed 0.022222222222222223
             TC           TN      TP           TK        C/N
0  19704.225352  1284.507042  1100.0  3021.126761  15.339912
********************
chicken manure
  mins_required=array([0.004201680672268907, 0.006896551724137931, 0.011834319526627219,
       0.0], dtype=object)
  min_required=0.011834319526627219 | maximum allowed 0.022222222222222223
             TC           TN      TP           TK        C/N
0  13795.321637  1141.520468  1100.0  2798.245614  12.085041
********************
soy meal
  mins_required=array([0.0028735632183908046, 0.003389830508474576, 0.01834862385321101,
       0.0], dtype=object)
  min_required=0.01834862385321101 | maximum allowed 0.022222222222222223
             TC           TN      TP           TK       C/N
0  17288.288288  1866.6666

In [18]:
sofix2 = {"TC": 10000, "TN": 800, "TP": 900, "TK": 2500, "C/N": 12.5}

calc_min_required(sofix1)

********************
cow manure
  mins_required=array([0.003246753246753247, 0.011764705882352941, 0.028985507246376812,
       0.0], dtype=object)
  min_required=0.028985507246376812 | maximum allowed 0.022222222222222223
             TC           TN      TP           TK        C/N
0  19704.225352  1284.507042  1100.0  3021.126761  15.339912
********************
chicken manure
  mins_required=array([0.004201680672268907, 0.006896551724137931, 0.011834319526627219,
       0.0], dtype=object)
  min_required=0.011834319526627219 | maximum allowed 0.022222222222222223
             TC           TN      TP           TK        C/N
0  13795.321637  1141.520468  1100.0  2798.245614  12.085041
********************
soy meal
  mins_required=array([0.0028735632183908046, 0.003389830508474576, 0.01834862385321101,
       0.0], dtype=object)
  min_required=0.01834862385321101 | maximum allowed 0.022222222222222223
             TC           TN      TP           TK       C/N
0  17288.288288  1866.6666

In [10]:
np.append(np.array([0.0, 1.0]), 0.9)

array([0. , 1. , 0.9])